In [ ]:
import tensorflow as tf
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
interpreter = tf.lite.Interpreter(model_path='lite-model_movenet_singlepose_lightning_3.tflite')
interpreter.allocate_tensors()

In [ ]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))

    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1)

0. Nose
1. Left Eye
2. Right Eye
3. Left Ear
4. Right Ear
5. Left Shoulder
6. Right Shoulder
7. Left Elbow
8. Right Elbow
9. Left Wrist
10. Right Wrist
11. Left Hip
12. Right Hip
13. Left Knee
14. **Right Knee**
15. Left Ankle
16. Right Ankle

In [ ]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [ ]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))

    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]

        if (c1 > confidence_threshold) & (c2 > confidence_threshold):
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)

In [ ]:
#cap = cv2.VideoCapture(0)


cap = cv2.VideoCapture('vtest.avi')  # Replace 'video_filename.mp4' with your video file's path

# Check if the video opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
else:
    while cap.isOpened():
        ret, frame = cap.read()

        # Reshape image
        img = frame.copy()
        img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 192,192)
        input_image = tf.cast(img, dtype=tf.float32)

        # Setup input and output
        input_details = interpreter.get_input_details()
        output_details = interpreter.get_output_details()

        # Make predictions
        interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
        interpreter.invoke()
        keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])

        # Rendering
        draw_connections(frame, keypoints_with_scores, EDGES, 0.4)
        draw_keypoints(frame, keypoints_with_scores, 0.4)
        from IPython.display import display, clear_output
        #Display the frame in the Colab notebook using IPython.display
        display_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        #display(display_frame)

        # Clear the previous frame for a smooth video display
        clear_output(wait=True)
        # Display the frame using matplotlib
        plt.imshow(display_frame)
        plt.axis('off')  # Turn off axis labels
        plt.show()

        # Find the indices of the right and left knee keypoints (if available)
        right_knee_index = None
        left_knee_index = None

        for idx, keypoint in enumerate(keypoints_with_scores[0]):
               x, y = keypoint[:2]  # Extract X and Y coordinates
               # Check if the coordinates are valid (non-negative values)
               if (x >= 0).all() and (y >= 0).all():

                  # Check the position of the keypoint to determine if it corresponds to the right or left knee
                  # You may need to adjust the position criteria based on your model's output
                  if (x > 90).all():  # Adjust the threshold as needed
                      right_knee_index = idx
                  else:
                      left_knee_index = idx

        # Print the right knee keypoint if it was found
        if right_knee_index is not None:
            right_knee_keypoint = keypoints_with_scores[0][right_knee_index]
            right_knee_x = right_knee_keypoint[1]
            right_knee_y = right_knee_keypoint[0]
            confidence = right_knee_keypoint[2]
            print(f"Right Knee Coordinates - X: {right_knee_x}, Y: {right_knee_y}")
            print(f"Confidence: {confidence}")
        else:
            print("Right knee not detected in this frame")

        # Print the left knee keypoint if it was found
        if left_knee_index is not None:
            left_knee_keypoint = keypoints_with_scores[0][left_knee_index]
            left_knee_x = left_knee_keypoint[1]
            left_knee_y = left_knee_keypoint[0]
            confidence = left_knee_keypoint[2]
            print(f"Left Knee Coordinates - X: {left_knee_x}, Y: {left_knee_y}")
            print(f"Confidence: {confidence}")
        else:
            print("Left knee not detected in this frame")




        # Press 'q' to exit the video playback
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [17]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()

    # Reshape image
    img = frame.copy()
    img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 192,192)
    input_image = tf.cast(img, dtype=tf.float32)

    # Setup input and output
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    # Make predictions
    interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
    interpreter.invoke()
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])

    # Rendering
    draw_connections(frame, keypoints_with_scores, EDGES, 0.4)
    draw_keypoints(frame, keypoints_with_scores, 0.4)

    cv2.imshow('MoveNet Lightning', frame)

    if cv2.waitKey(10) & 0xFF==ord('q'):
        break

cap.release()
cv2.destroyAllWindows()